In [ ]:
!pip install gradio

In [11]:
import cv2
import numpy as np
import gradio as gr
import matplotlib.pyplot as plt

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Load YOLO model
yolo_config = "/content/drive/My Drive/Object_Detection/yolov3.cfg"
yolo_weights = "/content/drive/My Drive/Object_Detection/yolov3.weights"
yolo_classes = "/content/drive/My Drive/Object_Detection/coco.names"

In [14]:
# Load class labels
with open(yolo_classes, "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [15]:
# Load YOLO model
net = cv2.dnn.readNet(yolo_weights, yolo_config)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [16]:
def detect_objects(image):
    img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert from RGB to BGR (OpenCV format)
    height, width, _ = img.shape

    # Convert image to blob
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Process detected objects
    class_ids, confidences, boxes = [], [], []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x, center_y, w, h = (detection[0:4] * [width, height, width, height]).astype("int")
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non-maximum suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    colors = np.random.uniform(0, 255, size=(len(classes), 3))

    # Draw bounding boxes
    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
        color = colors[class_ids[i]]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert back to RGB
    return img_rgb

In [18]:
# Gradio Interface
demo = gr.Interface(
                    fn=detect_objects,
                    inputs=gr.Image(type="numpy"),
                    outputs=gr.Image(type="numpy"),
                    title="YOLO Object Detection",
                    description="Upload an image to detect objects using YOLO v3.")
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5aea42c57a4664b164.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
